# Graphs, and the power of visualising datasets

## Introduction

In this pair of notebooks, we are going to show how drawing meaningful graphs of large datasets can highlight interesting features that might have otherwise been missed when looking line-by-line at the data. These two notebooks are intended as a starting point, showing examples of two fairly common graphs, with the goal that you can then apply the same techniques to your own data sets. The first of these two notebooks is going to gather a dataset from Discovery and manipulate it into pandas dataframe. The second notebook will then use the matplotlib library to draw the graphs. 

As always, we need to start by importing and installing required libraries. Note that this time, we are also gathering some pre-defined data - these are a couple of lists we are going to use to automate the calls to discovery to gather the data. The list of ship names is based on a list of ships of the Royal Navy from the War of 1812. The list of record series are some pre-selected series to limit our requests to Discovery.

In [ ]:
%pip install -q pandas
%pip install -q json
%pip install -q requests

import pandas as pd
import json
import requests

import aditional_data

record_series = aditional_data.admiralty_record_series
ship_list = aditional_data.ships

## What graphs are we going to draw?

Discovery has a lot of metadata about each record available to use, so we are going to need to narrow things down a bit. For the purpose of these notebooks, we are going to take the position of a researcher into record descriptions - record description length can vary significantly (from a word to [several paragraphs](https://discovery.nationalarchives.gov.uk/browse/r/r/C10488311), within related records). As the description, along with the title, is one of the main ways a typical user would find a record, we are going to gather data and draw graphs to help illuminate the area.

## Getting the data 

To get the data we need we are going to use the Discovery API, similar to the [main series of notebooks](../1-intro-to-discovery-api.ipynb). As we have previously gone into it in detail, we are going to do this in one cell. This cell is going to run through the lists we imported earlier, use their data to build a request to the `search` endpoint, and select only the data we need from the results. As we have previously decided that we are interested in record descriptions and covering dates, we are only going to keep fields relating to those. 

As this request is going take some time, it is worth considering how we want to structure the json we want to build from the response, to help ensure we get the right data first time. Given that we are going to be looping through a list of names of ships, and extracting data from each record in response, we are going to initially build a dictionary like this, which is structured in a way where we can see the result of each query:  

```json
[
    {
        "ship": "[ship name]",
        "data": [
            {
                "id": "[record id]",
                "title": "[record title]",
                "startDate": "[record start date]",
                "endDate": "[record end date]",
                "description": "[record description]",
                "reference": "[record reference]"
            },
            ...next record
        ]
    },
    ...next ship
]
```

 Note: with the length of the list of ship names, this cell makes a lot of requests to Discovery and can take a minute or two to run. Also note that as this is an example only, we're only retrieving the default number of records with each query; in a real world scenario, you would want to spend time refining the query to ensure you retrieve all intended records.

In [ ]:
ship_data = []

base_discovery_url = "https://discovery.nationalarchives.gov.uk/API/search/records?"

for ship in ship_list:
    url = base_discovery_url 
    for series in record_series:
        url += series
        url += "&"
    url += "sps.searchQuery=" + ship
    # print(url)                        # uncomment this line to see the url being used
    headers = {'Accept': 'application/json'}
    response = requests.get(url, headers=headers)
    response_json = response.json()
    #print(response_json)               # if you want to see the full response to each query, uncomment this line
    if response_json["records"] != []:
        found_data = []
        for record in response_json["records"]:
            found_data.append(
                {
                    "id": record["id"],
                    "title": record["title"],
                    "startDate": record["startDate"],
                    "endDate": record["endDate"],
                    "description": record["description"],
                    "reference": record["reference"].split("/")[0] # here, we would normally get a reference such as ADM 1/1234, but we only want the ADM 1 part
                }
            )
        ship_data.append(
            {
                "ship": ship,
                "data": found_data
            }
        )
        
        
print(json.dumps(ship_data, indent=4))

## Making a useful dataframe

A dataframe is the equivelent of a single sheet in a spreadsheet, and is the format of data we are going to use when drawing graphs. There are [three rules](https://byuidatascience.github.io/python4ds/tidy-data.html) for a useful and tidy dataframe:

1. Each variable forms a column
2. Each observation forms a row
3. Each value is a cell

For the data we have retrieved from Discovery, records act as equivalent to observations, and the variables are the metadata. The data from Discovery is not arranged exactly like this yet; but it is close, and the data structure we created when we ran the requests is going to make it easy to get it into the right structure. We can then perform some checks and modifications to make it easier to work with, such as ensuring that columns are all stored as the correct data type, and checking for missing data.

Note: we are showing one path to get the data into a dataframe, working with data from one endpoint from Discovery. This is a guide, not instructions, so when working with your own data, make sure you think through the steps you are going to take before you start.

We now have a large JSON file with all the data we want, as a result of the API calls in the previous cell. We also have a good understanding of what a nicely formatted dataframe looks like. The next stage is to use the `pandas` library to convert from the JSON to a dataframe. Doing so allows more powerful interactions with the dataframe, such as specifying the format of a column (e.g. that all values are dates), or filtering to only include rows that match a certain criteria.

The first step we are going to take is to flatten the json. Having a flat json file makes it very straightforwards to convert to a dataframe; each record relates to an item in the json and a row in the dataframe, with each key in the json relating to a column in the dataframe. As with before, it can be valuable to draw the structure of the json to help visualise the result and ensure the conversion runs correctly.



```json
[
    {
        "ship": "[ship name]",
        "id": "[record id]",
        "title": "[record title]",
        "startDate": "[record start date]",
        "endDate": "[record end date]",
        "description": "[record description]",
        "reference": "[record reference]"
    },
    ...next record
]
```

In [ ]:
ship_data_flat = []

for ship in ship_data:
    for record in ship["data"]:
        ship_data_flat.append(
            {
                "ship": ship["ship"],
                "id": record["id"],
                "title": record["title"],
                "startDate": record["startDate"],
                "endDate": record["endDate"],
                "description": record["description"],
                "reference": record["reference"]
            }
        )

print(json.dumps(ship_data_flat, indent=4))

Now that we have our flattened json, pandas makes it easy to convert it to a dataframe, as this next cell shows. We can then print the dataframe to see what it looks like.

In [ ]:
ship_data_frame = pd.DataFrame(ship_data_flat)

print(ship_data_frame)

## write to csv, even if the csv doesn't exist yet

ship_data_frame.to_csv("ship_data.csv")

### Post-dataframe checks and modifications

With the dataframe created, we can now inspect the date columns to ensure they are stored as the correct data type. Doing so will make it easier for us to do things like filtering by date, or calculating the difference between two dates. 

With pandas we can refer to a column in a dataframe using `dataframe['column name']`, similar to interacting with a dictionary. 

In [ ]:
ship_data_frame["startDate"] = pd.to_datetime(ship_data_frame["startDate"], dayfirst=True)
ship_data_frame["endDate"] = pd.to_datetime(ship_data_frame["endDate"], dayfirst=True)

Adding a new column is also easy, requiring a very similar syntax. Here, we are going to add a column that calculates the difference between the start and end dates of each record - a step now possible as we have ensured that the dates are stored as the correct data type. We're also going to add a column that maps a colour to each unique ship name, and for each record series, which will make it easier to add colour to the graphs. A final column will number each ship - this one will be useful instead for selecting subsets of data. 

We'll finish off these additions by printing the dataframe again, to see what it looks like now.

In [ ]:
ship_data_frame["record_duration"] = ship_data_frame["endDate"] - ship_data_frame["startDate"]

ship_colour_map = {}
for index, ship in enumerate(ship_list):
    ship_colour_map[ship] = f"C{index}"

ship_data_frame["ship_colour"] = ship_data_frame["ship"].map(ship_colour_map)

ship_number_map = {}
for index, ship in enumerate(ship_list):
    ship_number_map[ship] = index

ship_data_frame["ship_number"] = ship_data_frame["ship"].map(ship_number_map)

record_series_colour_map = {}
for index, series in enumerate(ship_data_frame["reference"].unique()):
    record_series_colour_map[series] = f"C{index}"

ship_data_frame["record_series_colour"] = ship_data_frame["reference"].map(record_series_colour_map)

ship_data_frame["description_length"] = ship_data_frame["description"].str.len()

print(ship_data_frame)

Checking for missing data is also easy, and can help us to identify any issues with the data. Here, we are going to check for missing data in the description column.

In [ ]:
## check for missing data in the description column

missing_cell_count = ship_data_frame["description"].isna().sum()

print("There are " + str(missing_cell_count) + " missing cells in the description column")

The final thing to do before we move onto the next notebook is to save the dataframe as a csv file. This is a very useful feature of pandas, as it makes the data highly portable - it can be opened in excel or other spreadsheet software, or a different python script (re-opening it with pandas). Here, we are going to use it to let us open the data in the next notebook. Note that the CSV format does not store the data type of each column, so we will need to ensure that we re-apply the data type changes we made in this notebook when we re-open the file. 

In [ ]:
## save the dataframe to a csv file

ship_data_frame.to_csv("ship_data.csv")